<a href="https://colab.research.google.com/github/pannettonne/Portfolio/blob/main/LaSalle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import importlib.util
import subprocess
#import pkg_resources
import sys
print("Empiezo")

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import os
import yagmail
from premailer import Premailer

#import pyjuice as juice
import css_inline

import re
import unicodedata
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

def minify_html(html: str) -> str:
    # elimina espacios/saltos entre etiquetas:  >   \n   <  ->  ><
    html = re.sub(r'>\s+<', '><', html)
    # colapsa espacios repetidos dentro de texto
    html = re.sub(r'\s{2,}', ' ', html)
    return html.strip()

def norm_text(s: str) -> str:
    # Normaliza unicode, quita NBSP y unifica guiones
    s = unicodedata.normalize("NFKC", s).replace("\xa0", " ")
    s = s.replace("–", "-").replace("—", "-")
    return " ".join(s.split())

os.environ['KEYRING_BACKEND'] = 'keyrings.alt.chainer.ChainerBackend'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from datetime import datetime

# Configuración para Google Colab
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
desired_capabilities = DesiredCapabilities.CHROME
desired_capabilities['goog:loggingPrefs'] = {'performance': 'ALL'}

# Configuración del WebDriver
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/chromium-browser'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

from webdriver_manager.chrome import ChromeDriverManager
service = Service(ChromeDriverManager().install())  # Use the automatically installed driver
driver = webdriver.Chrome(options=chrome_options)

def dump_debug(name="debug"):
    try:
        path_png = f"/tmp/{name}.png"
        path_html = f"/tmp/{name}.html"
        driver.save_screenshot(path_png)
        with open(path_html, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"[DEBUG] Screenshot: {path_png}")
        print(f"[DEBUG] HTML: {path_html}")
        sys.stdout.flush()
    except Exception as e:
        print(f"[DEBUG] No se pudo guardar debug: {e}", flush=True)

def cargar_vista(fecha_str: str, tipo: int, timeout: int = 25) -> str:
    """
    Llama a CargaDiv("div_padres","padre","padre_trabajo","fecha", fecha_str, "tipo", tipo)
    y devuelve el outerHTML de la tabla renderizada dentro de #div_padres.
    tipo=1 -> semanal, tipo=2 -> diaria
    """
    # Ejecuta la función JS del sitio con argumentos seguros
    driver.execute_script(
        'return CargaDiv("div_padres","padre","padre_trabajo","fecha", arguments[0], "tipo", arguments[1]);',
        fecha_str, tipo
    )
    # Espera a que aparezca un <th> con la fecha solicitada en #div_padres
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((
            By.XPATH, f'//*[@id="div_padres"]//th[contains(normalize-space(.), "{fecha_str}")]'
        ))
    )
    # Devuelve la tabla visible
    tabla_elem = driver.find_element(
        By.XPATH,
        '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
    )
    return tabla_elem.get_attribute("outerHTML")

try:
    # Navegar a la página de login
    driver.get('https://lasallesanrafael.sallenet.org')

    # Esperar a que la página cargue completamente
    time.sleep(3)

    # Encontrar los campos de entrada de usuario y contraseña
    username_field = driver.find_element(By.ID, 'username')
    password_field = driver.find_element(By.ID, 'password')

    # *** Manteniendo como lo tenías ***
    username_field.send_keys('davidpaniaguadominguez')
    password_field.send_keys('Valhalla99!')

    # Encontrar y hacer clic en el botón de login
    login_button = driver.find_element(By.ID, 'loginbtn')
    login_button.click()

    # Esperar a que la página de inicio cargue después del login
    time.sleep(10)

    # Verificar si el login fue exitoso (relajado)
    print("Login exitoso")

    # Navegar al módulo padres
    driver.get('https://lasallesanrafael.sallenet.org/mod/sallenet/modulos/padres/')
    time.sleep(10)  # Esperar a que la nueva página cargue completamente
    print("Navegación a la página de padres exitosa")

    # Entrar en "Trabajo"
    enlace_trabajo = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Trabajo')]"))
    )
    enlace_trabajo.click()

    # Esperar a que se renderice alguna tabla inicial (por si la vista por defecto tarda)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((
            By.XPATH, '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
        ))
    )

    # ---- Obtener fecha de hoy (Europe/Madrid preferida) ----
    try:
        from zoneinfo import ZoneInfo
        hoy_str = datetime.now(ZoneInfo("Europe/Madrid")).strftime("%d/%m/%Y")
    except Exception:
        hoy_str = datetime.utcnow().strftime("%d/%m/%Y")
    print(f"[DEBUG] Fecha solicitada: {hoy_str}", flush=True)

    # ---- 1) Vista SEMANAL ----
    print(f"[DEBUG] Solicitando VISTA SEMANAL para {hoy_str}", flush=True)
    tabla_semana_html = cargar_vista(hoy_str, tipo=1)
    print("[DEBUG] Vista semanal OK", flush=True)

    # ---- 2) Vista DIARIA ----
    print(f"[DEBUG] Solicitando VISTA DIARIA para {hoy_str}", flush=True)
    tabla_dia_html = cargar_vista(hoy_str, tipo=2)
    print("[DEBUG] Vista diaria OK", flush=True)

    # Preparar HTML para email
    tabla_html_modificada = tabla_dia_html        # DÍA ACTUAL
    tabla_html_modificada2 = tabla_semana_html    # SEMANA COMPLETA

    estilos_css = """
    <style>
        table { width: 100%; border-collapse: collapse; }
        th, td { border: 1px solid #dddddd; text-align: left; padding: 8px; vertical-align: top; }
        th { background-color: #f2f2f2; }
        tr:nth-child(even) { background-color: #f9f9f9; }
        h1 { margin: 0 0 10px; }
    </style>
    """

    try:
        # Crear instancia de Yagmail
        yag = yagmail.SMTP('elpaniaguado@gmail.com')

        # *** Manteniendo tus listas ***
        destinatarios3 = [
            'elpaniaguado@gmail.com',
            'cquinteromoreno@gmail.com',
            'sharonrdc@yahoo.es',
            'abdjimenez@hotmail.com',
            'paradinasmarcos@gmail.com',
            'paulaherediaalcantara@gmail.com',
            'mariofvilladangos@hotmail.com',
            'adrian.alvarez.munoz@gmail.com',
            'nu.abril@gmail.com',
            'lorenitavelayos@gmail.com',
            'judiith22@gmail.com',
            'aransay13@gmail.com',
            'emmaedani@gmail.com',
            'anaisabelmas@hotmail.com',
            'juandiazlorenzo@hotmail.com'
        ]
        destinatarios = ['elpaniaguado@gmail.com']  # ajusta cuál usas

        # Contenido del correo en HTML
        contenido_html = f"""
        <html>
        <head>{estilos_css}</head>
        <body>
            <h1 style="font-weight:bold;">DIA ACTUAL</h1>
            {tabla_html_modificada}
            <h1 style="font-weight:bold;">SEMANA COMPLETA</h1>
            {tabla_html_modificada2}
        </body>
        </html>
        """

        # Inline CSS + minificado
        contenido_html_inline = css_inline.inline(contenido_html)
        contenido_html_min = minify_html(contenido_html_inline)

        dia_actual_str = datetime.now().strftime("%d-%m-%Y")
        subject = "Calendario de tareas La Salle 1B " + dia_actual_str

        # Enviar el correo
        try:
            yag.send(
                to=destinatarios,  # o destinatarios3
                subject=subject,
                contents=[yagmail.raw(contenido_html_min)]
            )
            print("Correo enviado con éxito a todos los destinatarios.")
        except Exception as e:
            print(f"Error al enviar el correo: {e}")

    except Exception as e:
        print("Error al enviar el correo electrónico")
        print(e)

except Exception as e:
    print("Login fallido.", e)
    dump_debug("login_fallido")

finally:
    # Cerrar el navegador
    driver.quit()
    print("Fin.")



Empiezo


ModuleNotFoundError: No module named 'yagmail'